In [24]:
import sys
sys.path.insert(0, '/Users/gulugulu/repos/PuningAnalysis/src')
from scipy import stats

import numpy as np
from cogent3.maths.matrix_exponential_integration import expected_number_subs
from simulation.waiting_time_simulator_iid import get_histograms2
import numpy as np
import json
import plotly.express as px

from simulation.WTS_class import SeqSimulate



In [25]:
Q_matrix = np.array([[-1.75025094,  0.94143256,  0.45306226,  0.35575611],
       [ 0.49505807, -2.27893035,  0.88176788,  0.90210439],
       [ 0.36411798,  0.18070926, -0.91594839,  0.37112115],
       [ 0.77758124,  0.75396143,  0.52505328, -2.05659595]])
Q_dict = {'0': Q_matrix}
n = 1000
pi = [0.35, 0.15, 0.05, 0.45]
time_range = [0.5, 1, 2, 3, 4]
repeat_test = 100
time = 1

In [26]:
# simulator = SeqSimulate(Q_matrix, 100, 100, 5, pi)
# simulator.average_substitution(max_time=1)

In [27]:
2.1/60

0.035

In [28]:
elapsed_time = [0.035, 0.583, 2.3, 8.34, 19, 34, 54, 212]
sequence_length = [100, 500, 1000, 2000, 3000, 4000, 5000, 6000]

In [29]:
elapsed_time2 = [0.08, 3.27, 18.916, 125.4, 947]
sequence_length2 = [100, 500, 1000, 2000, 3000]

In [94]:
import plotly.express as px
import plotly.graph_objs as go

# Assuming you have these data lists: sequence_length, elapsed_time, elapsed_time_std
# And for a second set of data: sequence_length2, elapsed_time2, elapsed_time_std2

# Create a scatter plot with error bars
fig = go.Figure()

# Add first dataset
fig.add_trace(go.Scatter(
    x=sequence_length,
    y=elapsed_time,
    mode='markers+lines',  # Combines dot plot with connecting lines
    name='Updated WTS'
))

# Add second dataset
fig.add_trace(go.Scatter(
    x=sequence_length2,
    y=elapsed_time2,
    mode='markers+lines',
    name='Original WTS'
))

# Update layout with labels and title
fig.update_layout(
    title='Performance comparison between the old and updated WTS algorithm, t = 1, repeat = 100',
    xaxis_title='Sequence Length (bp)',
    yaxis_title='Elapsed Time (min)',
    legend_title=None,
    template='plotly_white'  # Optional: Provides a clean white background
)

# Show the plot
fig.show()

# Optionally save the plot as a PDF file
fig.write_image("elapsed_time_sequence_length.pdf")


## Test for different simulation time


In [31]:
theoretical_dict = {}
for i in range(len(time_range)):
    theoretical_dict[time_range[i]] = expected_number_subs(pi, Q_matrix, time_range[i])

theoretical_ns_error_dict = {time: 0 for time in time_range}

In [32]:

import json

def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    converted_data = {}
    for length, times in data.items():
        converted_data[length] = {float(key): value for key, value in times.items()}
    return converted_data

def compute_errors(ns_data, theoretical_dict):
    error_ns_dict = {}
    for key, times in ns_data.items():
        error_ns_dict[key] = {}
        for time, ns_value in times.items():
            ns_error_list = [a - theoretical_dict[time] for a in ns_value['ns_per_site_list']]
            error_ns_dict[key][time] = {
                'ns_per_site_list': ns_error_list,
                'avg_ns_per_site': ns_value['avg_ns_per_site'] - theoretical_dict[time]   
            }
    return error_ns_dict

from scipy import stats
import numpy as np

def calculate_p_values(error_ns_dict):
    p_values = []
    for length, times in error_ns_dict.items():
        for time, ns_values in times.items():
            ns_error_list = ns_values['ns_per_site_list']
            theoretical_value = 0  # Change if your theoretical value is different
            t_stat, p_value = stats.ttest_1samp(ns_error_list, theoretical_value)
            p_values.append(p_value)
    return p_values

In [33]:
import glob
import os

def find_json_files(directory, pattern):
    # Construct the full pattern with directory and pattern
    full_pattern = os.path.join(directory, pattern)
    # Use glob to find all files that match the full pattern
    file_paths = glob.glob(full_pattern)
    return file_paths

# Example usage
directory = '../../../results'

paths_3000 = find_json_files(directory, 'WTS_3000(*).json')
paths_2000 = find_json_files(directory,'WTS_2000(*).json')
paths_1000 = find_json_files(directory, 'WTS_1000(*).json')


### sequence length = 1000, repeat = 200

In [34]:
result_time_1000_dict = {}
for i in range(len(paths_1000)):
    result_time_1000_dict[i] = {}
    ns_result_time = load_data(paths_1000[i])
    error_ns_time = compute_errors(ns_result_time, theoretical_dict)
    p_values = calculate_p_values(error_ns_time)
    result_time_1000_dict[i] = {'ns_result_time': ns_result_time, 'error_ns_time': error_ns_time, 'p_values': p_values}

p_value_time_1000_all = np.array([result_time_1000_dict[0]['p_values'], result_time_1000_dict[1]['p_values'], result_time_1000_dict[2]['p_values']])

p_value_time_mean_1000 = np.mean(p_value_time_1000_all, axis=0)

# Calculate elementwise standard deviation
p_value_time_std_dev_1000 = np.std(p_value_time_1000_all, axis=0)

In [35]:
# Create a dot plot with error bars for standard deviations
fig_time_pvalue = px.scatter(x=time_range, y=p_value_time_mean_1000,
                             labels={'x': 'Simulation Time', 'y': 'P-value'},
                             title='P-value for Different Simulation Time, sequence length 1000',
                             error_y=p_value_time_std_dev_1000)

fig_time_pvalue.add_scatter(x=time_range, y=p_value_time_mean_1000, mode='lines', showlegend=False)

# Add a horizontal line at y=0.05
fig_time_pvalue.add_scatter(x=[0, 5], y=[0.05, 0.05],
                              mode='lines', line=dict(color='red', dash='dash', width=3),
                              name='Threshold: p-value = 0.05')
# Save the plot as a PDF
# fig_repeat_pvalue.write_image("P_value_length_time.pdf")
fig_time_pvalue.update_yaxes(range=[0, 1])
fig_time_pvalue.update_xaxes(range=[0, 5])

# Show the plot
fig_time_pvalue.show()

### Test for different simulation time, sequence length = 2000, repeat = 200

In [36]:
result_time_2000_dict = {}
for i in range(len(paths_2000)):
    result_time_2000_dict[i] = {}
    ns_result_time = load_data(paths_2000[i])
    error_ns_time = compute_errors(ns_result_time, theoretical_dict)
    p_values = calculate_p_values(error_ns_time)
    result_time_2000_dict[i] = {'ns_result_time': ns_result_time, 'error_ns_time': error_ns_time, 'p_values': p_values}

p_value_time_2000_all = np.array([result_time_2000_dict[0]['p_values'], result_time_2000_dict[1]['p_values'], result_time_2000_dict[2]['p_values']])

p_value_time_mean_2000 = np.mean(p_value_time_2000_all, axis=0)

# Calculate elementwise standard deviation
p_value_time_std_dev_2000 = np.std(p_value_time_2000_all, axis=0)

In [37]:
# Create a dot plot with error bars for standard deviations
fig_time_pvalue = px.scatter(x=time_range, y=p_value_time_mean_2000,
                             labels={'x': 'Simulation Time', 'y': 'P-value'},
                             title='P-value for Different Simulation Time, sequence length = 2000',
                              error_y=p_value_time_std_dev_2000
                            )

fig_time_pvalue.add_scatter(x=time_range, y=p_value_time_mean_2000, mode='lines', showlegend=False)

# Add a horizontal line at y=0.05
fig_time_pvalue.add_scatter(x=[0, 5], y=[0.05, 0.05],
                              mode='lines', line=dict(color='red', dash='dash', width=3),
                              name='Threshold: p-value = 0.05')
# Save the plot as a PDF
# fig_repeat_pvalue.write_image("P_value_length_time.pdf")
fig_time_pvalue.update_yaxes(range=[0, 1])
fig_time_pvalue.update_xaxes(range=[0, 5])

# Show the plot
fig_time_pvalue.show()

### Length = 3000, repeat  = 100

In [38]:
result_time_2000_dict = {}
for i in range(len(paths_2000)):
    result_time_2000_dict[i] = {}
    ns_result_time = load_data(paths_2000[i])
    error_ns_time = compute_errors(ns_result_time, theoretical_dict)
    p_values = calculate_p_values(error_ns_time)
    result_time_2000_dict[i] = {'ns_result_time': ns_result_time, 'error_ns_time': error_ns_time, 'p_values': p_values}



In [39]:
result_time_3000_dict = {}
for i in range(len(paths_3000)):
    result_time_3000_dict[i] = {}
    ns_result_time = load_data(paths_3000[i])
    error_ns_time = compute_errors(ns_result_time, theoretical_dict)
    p_values = calculate_p_values(error_ns_time)
    result_time_3000_dict[i] = {'ns_result_time': ns_result_time, 'error_ns_time': error_ns_time, 'p_values': p_values}


p_value_time_3000_all = np.array([result_time_3000_dict[0]['p_values'], result_time_3000_dict[1]['p_values'], result_time_3000_dict[2]['p_values']])

p_value_time_mean_3000 = np.mean(p_value_time_3000_all, axis=0)

# Calculate elementwise standard deviation
p_value_time_std_dev_3000 = np.std(p_value_time_3000_all, axis=0)






In [40]:
# Create a dot plot with error bars for standard deviations
fig_time_pvalue = px.scatter(x=time_range, y=p_value_time_mean_3000,
                             labels={'x': 'Simulation Time', 'y': 'P-value'},
                             title='P-value for Different Simulation Time, sequence length = 3000',
                              error_y=p_value_time_std_dev_3000
                            )

fig_time_pvalue.add_scatter(x=time_range, y=p_value_time_mean_3000, mode='lines', showlegend=False)

# Add a horizontal line at y=0.05
fig_time_pvalue.add_scatter(x=[0, 5], y=[0.05, 0.05],
                              mode='lines', line=dict(color='red', dash='dash', width=3),
                              name='Threshold: p-value = 0.05')
# Save the plot as a PDF
# fig_repeat_pvalue.write_image("P_value_length_time.pdf")
fig_time_pvalue.update_yaxes(range=[0, 1])
fig_time_pvalue.update_xaxes(range=[0, 5])

# Show the plot
fig_time_pvalue.show()

In [41]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming time_range and p_value_time_mean/std_dev are already defined
# Define the sequence lengths you will use
seq_lengths = [1000, 2000, 3000]

# Create a subplot figure with 3 rows
fig = make_subplots(rows=3, cols=1,
                    vertical_spacing=0.1)

# Dictionary to store data, replace this with your actual data fetching logic
data = {
    1000: {'mean': p_value_time_mean_1000, 'std_dev': p_value_time_std_dev_1000},
    2000: {'mean': p_value_time_mean_2000, 'std_dev': p_value_time_std_dev_2000},
    3000: {'mean': p_value_time_mean_3000, 'std_dev': p_value_time_std_dev_3000}
}

# Loop through each sequence length and add a trace for each one
for i, seq_len in enumerate(seq_lengths, start=1):
    fig.add_trace(
        go.Scatter(
            x=time_range, 
            y=data[seq_len]['mean'], 
            mode='markers+lines',
            error_y=dict(type='data', array=data[seq_len]['std_dev']),
            name=f'Sequence Length = {seq_len}'
        ),
        row=i, col=1
    )

    # Add a horizontal line at y=0.05 to indicate significance threshold
    fig.add_trace(
        go.Scatter(
            x=[0, 5], y=[0.05, 0.05], mode='lines',
            line=dict(color='red', dash='dash', width=3),
            name='Threshold: p-value = 0.05',
            showlegend=(i == 1)  # Show legend only in the first subplot
        ),
        row=i, col=1
    )

# Update y-axis and x-axis properties for all plots
fig.update_yaxes(title_text="P-value", range=[0, 1])
fig.update_xaxes(title_text="Simulation Time", row=3, col=1)

# Update layout and titles
fig.update_layout(
    title='P-value for Different Simulation Times Across Sequence Lengths',
    height=900,  # Adjust height to fit all subplots comfortably
    showlegend=True
)

# Show the plot
fig.show()


In [42]:
data

{1000: {'mean': array([0.75788015, 0.52934361, 0.49346531, 0.47017435, 0.55685177]),
  'std_dev': array([0.08327791, 0.18956336, 0.25840118, 0.33190031, 0.38201305])},
 2000: {'mean': array([0.59547294, 0.42064619, 0.54975557, 0.14945697, 0.65697185]),
  'std_dev': array([0.33018017, 0.22105964, 0.32148217, 0.17270346, 0.08368703])},
 3000: {'mean': array([0.27011924, 0.41739007, 0.37502082, 0.38777774, 0.33712003]),
  'std_dev': array([0.09626923, 0.08949583, 0.07571595, 0.20546317, 0.38083964])}}

In [43]:
error_ns_dict_time_length = {'1000': result_time_1000_dict[0]['error_ns_time']['1000'], '2000': result_time_2000_dict[0]['error_ns_time']['2000'], '3000': result_time_3000_dict[0]['error_ns_time']['3000']}


In [44]:
error_fig = get_histograms2(error_ns_dict_time_length, theoretical_ns_error_dict)
error_fig.update_layout(
    title_text="Number of Subsitituion Error per Site with Average and Theoretical Values",
    showlegend=False)
error_fig.update_xaxes(range=[-0.2, 0.25])
error_fig.update_yaxes(range=[0, 0.4])

### Test for different repeat, length = 1000, time = 1

In [45]:

with open('../../../results/WTS_repeat.json', 'r') as file:
    repeat_data = json.load(file)

with open('../../../results/WTS_repeat(2).json', 'r') as file:
    repeat_data2 = json.load(file)

with open('../../../results/WTS_repeat(3).json', 'r') as file:
    repeat_data3 = json.load(file)

In [46]:
repeats = [10, 50, 100, 200, 300, 500]

p_value_list_repeat = []
for key, value in repeat_data.items():
    p_value_list_repeat.append(value['p_value'])

p_value_list_repeat2 = []
for key, value in repeat_data2.items():
    p_value_list_repeat2.append(value['p_value'])

p_value_list_repeat3 = []
for key, value in repeat_data3.items():
    p_value_list_repeat3.append(value['p_value'])

In [47]:
p_value_repeat_total = np.array([p_value_list_repeat, p_value_list_repeat2, p_value_list_repeat3])
# Calculate elementwise mean
p_value_repeat_mean_values = np.mean(p_value_repeat_total, axis=0)

# Calculate elementwise standard deviation
p_value_repeat_std_dev_values = np.std(p_value_repeat_total, axis=0)

In [48]:
# Create a dot plot with error bars for standard deviations
fig_repeat_pvalue = px.scatter(x=repeats, y=p_value_repeat_mean_values,
                             labels={'x': 'Repetition', 'y': 'P-value'},
                             title='P-value for Different Repetitions',
                             error_y=p_value_repeat_std_dev_values)

fig_repeat_pvalue.add_scatter(x=repeats, y=p_value_repeat_mean_values, mode='lines', showlegend=False)

# Add a horizontal line at y=0.05
fig_repeat_pvalue.add_scatter(x=[0, 550], y=[0.05, 0.05],
                              mode='lines', line=dict(color='red', dash='dash', width=3),
                              name='Threshold: p-value = 0.05')
# Save the plot as a PDF
# fig_repeat_pvalue.write_image("P_value_length_time.pdf")
fig_repeat_pvalue.update_yaxes(range=[0, 1.1])
fig_repeat_pvalue.update_xaxes(range=[0, 550])

# Show the plot
fig_repeat_pvalue.show()

In [49]:
error_ns_dict_repeat_1 = {}
for key, values in repeat_data.items():
    error_ns_dict_repeat_1[key] = {}  # Ensures key is initialized
    ns_error_list = [a - values['theoretical_value'] for a in values['ns_per_site_list']]
    error_ns_dict_repeat_1[key] = {
        'ns_per_site_list': ns_error_list,
        'avg_ns_per_site': values['avg_ns_per_site'] - values['theoretical_value'],
        'theoretical_value': 0
        }
    
error_ns_dict_repeat_2 = {}
for key, values in repeat_data2.items():
    error_ns_dict_repeat_2[key] = {}  # Ensures key is initialized
    ns_error_list = [a - values['theoretical_value'] for a in values['ns_per_site_list']]
    error_ns_dict_repeat_2[key] = {
        'ns_per_site_list': ns_error_list,
        'avg_ns_per_site': values['avg_ns_per_site'] - values['theoretical_value'],
        'theoretical_value': 0
        }

error_ns_dict_repeat_3 = {}
for key, values in repeat_data3.items():
    error_ns_dict_repeat_3[key] = {}  # Ensures key is initialized
    ns_error_list = [a - values['theoretical_value'] for a in values['ns_per_site_list']]
    error_ns_dict_repeat_3[key] = {
        'ns_per_site_list': ns_error_list,
        'avg_ns_per_site': values['avg_ns_per_site'] - values['theoretical_value'],
        'theoretical_value': 0
        }

In [50]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_ns_per_site_histograms(data):
    """
    Plot histograms for ns_per_site_list with vertical lines for avg_ns_per_site and theoretical_value.
    
    Parameters:
        data (dict): A dictionary with keys as repetition numbers and values as another dict
                     containing ns_per_site_list, avg_ns_per_site, and theoretical_value.
    """
    # Create a subplot for each key
    rows = len(data)
    fig = make_subplots(rows=rows, cols=1)

    for i, (key, values) in enumerate(data.items(), start=1):
        # Add histogram to subplot
        fig.add_trace(
            go.Histogram(x=values['ns_per_site_list'], histnorm='probability',
                    marker=dict(line=dict(width=1)), xbins=dict(  # Control the bar widths here
                        # start=x_min,
                        # end=x_max,
                        size=0.005
                    ),
                name=f'Ns per Site - Rep {key}'),
                row=i, col=1)

        # Add vertical lines for average and theoretical values
        fig.add_vline(x=values['avg_ns_per_site'], line_color='blue', line_dash='dash', row=i, col=1,
                      annotation_text='Avg', annotation_position='top right')
        fig.add_vline(x=values['theoretical_value'], line_color='red', line_dash='dash', row=i, col=1,
                      annotation_text='Theoretical', annotation_position='top left')


        

    # Update layout
    fig.update_layout(
        height=300 * rows,  # Adjust height based on number of subplots
        width=800,
        title_text="Histograms of Ns per Site with Average and Theoretical Values",
        showlegend=False)
    
    return fig

In [51]:
fig = plot_ns_per_site_histograms(error_ns_dict_repeat_3)
fig.update_xaxes(range=[-0.15, 0.15])
fig.show()

In [52]:
data_10 = repeat_data['10']['ns_per_site_list']
data_50 = repeat_data['50']['ns_per_site_list']
data_100 = repeat_data['100']['ns_per_site_list']
data_200 = repeat_data['200']['ns_per_site_list']
data_300 = repeat_data['300']['ns_per_site_list']
data_500 = repeat_data['500']['ns_per_site_list']



In [53]:
import numpy as np
from scipy.stats import norm, kstest

# Generate a sample from a normal distribution
theoretcal = expected_number_subs(pi, Q_matrix, 1)
mean = theoretcal
variance = theoretcal/2000
std_dev = np.sqrt(variance)

# Perform the K-S test comparing the data against the theoretical normal CDF
ks_statistic, p_value = kstest(data_100, 'norm', args=(mean, std_dev))

print("K-S Statistic:", ks_statistic)
print("P-Value:", p_value)

# Interpret the result
if p_value < 0.05:
    print("Reject the null hypothesis - The sample does not follow the normal distribution.")
else:
    print("Fail to reject the null hypothesis - The sample follows the normal distribution.")


K-S Statistic: 0.10819697915608428
P-Value: 0.17870097661028753
Fail to reject the null hypothesis - The sample follows the normal distribution.


In [54]:
import numpy as np
from scipy.stats import norm, kstest

# Generate a sample from a normal distribution
theoretcal = expected_number_subs(pi, Q_matrix, 1)
mean = theoretcal
variance = theoretcal/2000
std_dev = np.sqrt(variance)

# Perform the K-S test comparing the data against the theoretical normal CDF
ks_statistic, p_value = kstest(data_100, 'norm', args=(mean, std_dev))

print("K-S Statistic:", ks_statistic)
print("P-Value:", p_value)

# Interpret the result
if p_value < 0.05:
    print("Reject the null hypothesis - The sample does not follow the normal distribution.")
else:
    print("Fail to reject the null hypothesis - The sample follows the normal distribution.")

K-S Statistic: 0.10819697915608428
P-Value: 0.17870097661028753
Fail to reject the null hypothesis - The sample follows the normal distribution.


In [55]:
theoretcal = 2
mean = theoretcal
variance = theoretcal/1000
std_dev = np.sqrt(variance)

n = np.square(((1.96*std_dev)/0.01))
n

76.832

In [56]:
std_dev

0.044721359549995794

## Changing Q and pi


In [57]:
paths_Q = find_json_files(directory, 'WTS_Q(*).json')
result_Q_dict = {}
for i in range(len(paths_Q)):
    with open(paths_Q[i], 'r') as file:
        ns_result_Q = json.load(file)
    result_Q_dict[i] = ns_result_Q

error_Q_dict = {}
p_value_Q_dict = {}
for repeat, value in result_Q_dict.items():
    error_Q_dict[repeat] = {}
    p_value_Q_dict[repeat] = {}
    for Q, data in value.items():
        error_Q_dict[repeat][Q] = data['error']
        p_value_Q_dict[repeat][Q] = data['p_value']

In [58]:
paths_pi = find_json_files(directory, 'WTS_pi(*).json')
result_pi_dict = {}
for i in range(len(paths_pi)):
    with open(paths_pi[i], 'r') as file:
        ns_result_pi = json.load(file)
    result_pi_dict[i] = ns_result_pi

error_pi_dict = {}
p_value_pi_dict = {}
for repeat, value in result_pi_dict.items():
    error_pi_dict[repeat] = {}
    p_value_pi_dict[repeat] = {}
    for pi, data in value.items():
        error_pi_dict[repeat][pi] = data['error']
        p_value_pi_dict[repeat][pi] = data['p_value']


## Stability test ANOVA

In [59]:
ns_result_length_time1 = {'1000': result_time_1000_dict[0]['ns_result_time']['1000'], '2000': result_time_2000_dict[0]['ns_result_time']['2000'], '3000': result_time_3000_dict[0]['ns_result_time']['3000']}
ns_result_time = {'0': result_time_1000_dict[0]['ns_result_time']['1000'], '1': result_time_1000_dict[1]['ns_result_time']['1000'], '2': result_time_1000_dict[2]['ns_result_time']['1000']}
ns_result_length_time2 = {'1000': result_time_1000_dict[1]['ns_result_time']['1000'], '2000': result_time_2000_dict[1]['ns_result_time']['2000'], '3000': result_time_3000_dict[1]['ns_result_time']['3000']}
ns_result_length_time3 = {'1000': result_time_1000_dict[2]['ns_result_time']['1000'], '2000': result_time_2000_dict[2]['ns_result_time']['2000'], '3000': result_time_3000_dict[2]['ns_result_time']['3000']}


In [60]:
time_lenth_cv1 = {}
for length, times in ns_result_length_time1.items():
    time_lenth_cv1[length] = {}
    for time, data in times.items():
        ns_per_site_list = data['ns_per_site_list']
        avg_ns_per_site = data['avg_ns_per_site']
        std_dev = np.std(ns_per_site_list)
        cv = std_dev/avg_ns_per_site
        time_lenth_cv1[length][time] = cv*100

time_lenth_cv2 = {}
for length, times in ns_result_length_time2.items():
    time_lenth_cv2[length] = {}
    for time, data in times.items():
        ns_per_site_list = data['ns_per_site_list']
        avg_ns_per_site = data['avg_ns_per_site']
        std_dev = np.std(ns_per_site_list)
        cv = std_dev/avg_ns_per_site
        time_lenth_cv2[length][time] = cv*100

time_lenth_cv3 = {}
for length, times in ns_result_length_time3.items():
    time_lenth_cv3[length] = {}
    for time, data in times.items():
        ns_per_site_list = data['ns_per_site_list']
        avg_ns_per_site = data['avg_ns_per_site']
        std_dev = np.std(ns_per_site_list)
        cv = std_dev/avg_ns_per_site
        time_lenth_cv3[length][time] = cv*100

time_lenth_cv = {0: time_lenth_cv1, 1: time_lenth_cv2, 2:time_lenth_cv3}



In [61]:
length_cv = {}
for repeat, element in time_lenth_cv.items():
    length_cv[repeat] = {}
    for length, time in element.items():
        length_cv[repeat][length] = element[length][1]

In [62]:
time_cv = {}
for repeat, times in ns_result_time.items():
    time_cv[repeat] = {}
    for time, data in times.items():
        ns_per_site_list = data['ns_per_site_list']
        avg_ns_per_site = data['avg_ns_per_site']
        std_dev = np.std(ns_per_site_list)
        cv = std_dev/avg_ns_per_site
        time_cv[repeat][time] = cv*100

In [63]:
Q_cv = {}
for repeat, value in result_Q_dict.items():
    Q_cv[repeat] = {}
    for Q, data in value.items():
        Q_cv[repeat][Q] = data['cv']*100

pi_cv = {}
for repeat, value in result_pi_dict.items():
    pi_cv[repeat] = {}
    for pi, data in value.items():
        pi_cv[repeat][pi] = data['cv']*100

time_cv_list = [value for nested_dict in time_cv.values() for value in nested_dict.values()]
length_cv_list = [value for nested_dict in length_cv.values() for value in nested_dict.values()]
Q_cv_list = [value for nested_dict in Q_cv.values() for value in nested_dict.values()]
pi_cv_list = [value for nested_dict in pi_cv.values() for value in nested_dict.values()]


In [64]:
# import pandas as pd
# from scipy import stats
# Q_matrix1 = np.array([[-1.75025094,  0.94143256,  0.45306226,  0.35575611],
#        [ 0.49505807, -2.27893035,  0.88176788,  0.90210439],
#        [ 0.36411798,  0.18070926, -0.91594839,  0.37112115],
#        [ 0.77758124,  0.75396143,  0.52505328, -2.05659595]])

# Q_matrix2 = np.array([[-0.8913651590062768,
#   0.06758713003981928,
#   0.26750861021623484,
#   0.5562694187502226],
#  [0.48766534934241146,
#   -1.1914228729789795,
#   0.2796236707498311,
#   0.42413385288673694],
#  [0.48963798725695706,
#   0.6855172408178508,
#   -1.5509279536481513,
#   0.3757727255733433],
#  [0.1537313911769212,
#   0.7977890102842006,
#   0.551525454894267,
#   -1.5030458563553888]])

# pi1 = [0.35, 0.15, 0.05, 0.45]
# pi2 = [0.1, 0.1, 0.1, 0.7]

# lengths = [1000, 2000]
# times = [0.5, 2]
# Q_matrices = [Q_matrix1, Q_matrix2]
# pis = [pi1, pi2]
# repeat = 100
# seeds = range(16)
# import itertools
# param_combinations = list(itertools.product(lengths, times, Q_matrices, pis))
# results = []

# # Running simulations for each combination
# for i in range(len(param_combinations)):
#     length, time, Q, pi = param_combinations[i]
#     simulator = SeqSimulate(Q, length, 100, seeds[i], pi)
#     ns_per_site, avg_ns_per_site = simulator.average_substitution(max_time=time)
#     std = np.std(ns_per_site)
#     cv = std / avg_ns_per_site
#     theoretical = expected_number_subs(pi, Q, time)
#     error_list = [a - theoretical for a in ns_per_site]
#     t_stat, p_value = stats.ttest_1samp(ns_per_site, theoretical)

#     # Append results
#     results.append({
#         'Length': length,
#         'Time': time,
#         'Q': Q,
#         'Pi': pi,
#         'Seed': seeds[i],
#         'Avg_NS_Per_Site': avg_ns_per_site,
#         'Theoretical_Value': theoretical,
#         'Error': error_list,
#         'P_Value': p_value,
#         'T_Stat': t_stat,
#         'CV': cv,
#         'Standard_Deviation': std
#     })

# # Convert to DataFrame for analysis and reporting
# results_df = pd.DataFrame(results)

In [65]:
# with open('../../../results/anova_test.json', 'w') as outfile:
#     json.dump(result_no_Q_value, outfile, indent=4)

In [66]:
with open('../../../results/anova_test.json', 'r') as file:
    anova_results = json.load(file)

cv = []
for element in anova_results:
    cv.append(element['CV'])

In [67]:
# import pandas as pd
# import statsmodels.api as sm
# from statsmodels.formula.api import ols
# # Example data
# data = {
#     'CV': cv,
#     'Sequence_Length': ['long', 'long', 'long', 'long', 'long', 'long', 'long', 'long', 'short', 'short', 'short', 'short','short', 'short', 'short', 'short'],
#     'Simulation_Time': ['short', 'short', 'short',  'short', 'long', 'long', 'long', 'long', 'short', 'short',  'short', 'short', 'long', 'long', 'long', 'long'],
#     'Q': ['type1', 'type1', 'type2', 'type2', 'type1', 'type1', 'type2', 'type2', 'type1', 'type1', 'type2', 'type2', 'type1', 'type1', 'type2', 'type2'],
#     'Pi': ['dist1', 'dist2', 'dist1', 'dist2', 'dist1', 'dist2', 'dist1', 'dist2','dist1', 'dist2', 'dist1', 'dist2', 'dist1', 'dist2', 'dist1', 'dist2']
# }
# df = pd.DataFrame(data)

# # Fit ANOVA model with interaction terms if needed
# model = ols('CV ~ C(Sequence_Length) + C(Simulation_Time) + C(Q) + C(Pi)', data=df).fit()
# anova_results = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA for unbalanced designs

# # Print the ANOVA table
# print(anova_results)

In [93]:
import plotly.express as px
import pandas as pd

# Example DataFrame setup
data = {
    'CV': time_cv_list + length_cv_list + Q_cv_list + pi_cv_list,
    'ParameterGroup': ['Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time',
                       'Length', 'Length', 'Length', 'Length', 'Length', 'Length', 'Length', 'Length', 'Length',
                       'Q', 'Q', 'Q', 'Q', 'Q', 'Q', 'Q', 'Q', 'Q',
                       'pi', 'pi', 'pi', 'pi', 'pi', 'pi', 'pi', 'pi', 'pi']
}
df = pd.DataFrame(data)

# Create a box plot
fig = px.box(df, x='ParameterGroup', y='CV',
             labels={'CV': 'Coefficient of Variation (%)', 'ParameterGroup': 'Parameter Group'},
             color='ParameterGroup',  # Adds different colors per ParameterGroup
             template='plotly_white'  # Clean white background
             )

# Add a horizontal line at y=5 with annotation
fig.add_hline(y=5, line_color='red', line_dash='dash',
              annotation_text='Threshold', annotation_position='top left')

# Update the y-axis range to focus on the data
fig.update_yaxes(range=[0, 5.5])  # Adjust based on your maximum CV value

# Optionally enhance the layout further
fig.update_layout(plot_bgcolor='white',  # Ensures background remains white across the plot
    yaxis_title='Coefficient of Variation (%)',
    xaxis_title='Parameter Group Variation',
    showlegend=False
)

# Show the plot
fig.show()

# Optionally, save the plot as a PDF file
fig.write_image("parameter_variation_impact.pdf")


/Users/gulugulu/opt/anaconda3/envs/c312/lib/python3.12/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [69]:
p_value_time_list = [list(a) for a in p_value_time_1000_all]


In [70]:
p_value_time_length_all = [p_value_time_1000_all, p_value_time_2000_all, p_value_time_3000_all]
p_value_length_list = []
for element in p_value_time_length_all:
    p_value_length_list.append(list(element[:,0]))

In [71]:
p_value_pi_list = []
for repeat, pi in p_value_pi_dict.items():
    p_value_pi_list.append(list(pi.values()))

p_value_Q_list = []
for repeat, Q in p_value_Q_dict.items():
    p_value_Q_list.append(list(Q.values()))


In [82]:
p_value_pi = [item for sublist in p_value_pi_list for item in sublist]
p_value_time = [item for sublist in p_value_time_list for item in sublist]
p_value_length = [item for sublist in p_value_length_list for item in sublist]
p_value_Q = [item for sublist in p_value_Q_list for item in sublist]

In [92]:
import plotly.express as px
import pandas as pd

# Example DataFrame setup
data = {
    'p-value':  p_value_time + p_value_length + p_value_Q + p_value_pi,
    'ParameterGroup': ['Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time', 'Time','Time', 'Time',
                       'Length', 'Length', 'Length', 'Length', 'Length', 'Length', 'Length', 'Length','Length',
                       'Q', 'Q', 'Q', 'Q', 'Q', 'Q', 'Q', 'Q', 'Q',
                       'pi', 'pi', 'pi', 'pi', 'pi', 'pi', 'pi', 'pi', 'pi'
                       ]
}
df = pd.DataFrame(data)

# Create a box plot
fig = px.box(df, x='ParameterGroup', y='p-value',
             labels={'p-value': 'P - value ', 'ParameterGroup': 'Parameter Group'},
             color='ParameterGroup',  # Adds different colors per ParameterGroup
             template='plotly_white'  # Clean white background
             )

# Add a horizontal line at y=5 with annotation
fig.add_hline(y=0.05, line_color='red', line_dash='dash',
              annotation_text='Threshold', annotation_position='top left')

# Update the y-axis range to focus on the data
fig.update_yaxes(range=[-0.1, 1.1])  # Adjust based on your maximum CV value

# Optionally enhance the layout further
fig.update_layout(plot_bgcolor='white',  # Ensures background remains white across the plot
    yaxis_title='P-value',
    xaxis_title='Parameter Group Variation',
    showlegend=False
)

# Show the plot
fig.show()

# Optionally, save the plot as a PDF file
fig.write_image("parameter_accuracy_impact.pdf")

/Users/gulugulu/opt/anaconda3/envs/c312/lib/python3.12/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

